In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import openai

from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv('OPENAI_KEY')

# First experiment

In [ ]:
from utils.constants import STAGE_DIR

with open(STAGE_DIR / '01_sentences.txt', 'r') as f:
    sentences = f.readlines()

In [ ]:
sentences = '''
Los trabajadores tienen como derechos básicos, con el contenido y alcance que para cada uno de los mismos disponga su específica normativa, los de Negociación colectiva

Los trabajadores menores de dieciocho años no podrán realizar trabajos nocturnos ni aquellas actividades o puestos de trabajo respecto a los que se establezcan limitaciones a su contratación conforme a lo dispuesto en la Ley 31/1995, de 8 de noviembre, de Prevención de Riesgos Laborales, y en las normas reglamentarias aplicables

Si el representante legal de una persona de capacidad limitada la autoriza expresa o tácitamente para realizar un trabajo, queda esta también autorizada para ejercitar los derechos y cumplir los deberes que se derivan de su contrato y para su cesación

Podrán contratar la prestación de su trabajo Los extranjeros, de acuerdo con lo dispuesto en la legislación específica sobre la materia

El contrato de trabajo se podrá celebrar por escrito o de palabra
'''

prompt = f'''
Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to develop a spanish event extraction task. The steps to achieve it are the following:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger, usually in the form of a verb. It can happen that a sentence relates more than one object entity with the same subject and event trigger. The sentences can contain entities and phrases that don't correspond to any classification. Also, the object can be separated from the subject and event trigger by these complements. The definitions of each category are the following:
    * event trigger: It refers to the action that is enforced by the legal text. It can be in a negative form, e.g. "no pueden".
    * subject entity: It refers to the entity that most follow, do the action of the event trigger.
    * object entity: It refers to the entity that is the receptor of the action. In the legal domain, it can be, for example, a right, a beneficiary from the action, an institution, a non-right, etc. 
    
    Example 1:
    Input: Cuando la relación laboral sea de duración superior a cuatro semanas, el empresario deberá informar por escrito, en los términos y plazos que se establezcan reglamentariamente, al trabajador sobre los elementos esenciales del contrato y las principales condiciones de ejecución de la prestación laboral, siempre que tales elementos y condiciones no figuren en el contrato de trabajo formalizado por escrito.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá informar por escrito"
        
    Example 2:
    Input: La denegación de la solicitud deberá ser notificada por el empresario al trabajador por escrito y de manera motivada.
    Output:
        * subject entity: "empresario"
        * object entity: "trabajador"
        * event trigger: "deberá ser notificada"

3. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    - sentence: the sentence
    - subject: the subject entity
    - object: the object entity
    - event: the event trigger
Note that it can be repeated sentences because of the different subject-event-object combinations.

The set of sentences in spanish to use is the following:

```
{sentences}
```
'''

In [ ]:
response = get_completion(prompt)

In [ ]:
print(response)

# Experiments with `PlanTL-GOB-ES/RoBERTalex`

In [ ]:
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, FillMaskPipeline
from pprint import pprint
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"¡Hola <mask>!"
res_hf = pipeline(text)
pprint([r['token_str'] for r in res_hf])

In [ ]:
tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
model.eval()
pipeline = FillMaskPipeline(model, tokenizer_hf)
text = f"El contrato de trabajo se podrá <mask>"
res_hf = pipeline(text)

pprint([r['token_str'] for r in res_hf])

In [ ]:
print(res_hf)

In [ ]:
seed = f"El contrato de trabajo se podrá <mask>"
i = 10
while i:
    tokenizer_hf = AutoTokenizer.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model = AutoModelForMaskedLM.from_pretrained('PlanTL-GOB-ES/RoBERTalex')
    model.eval()
    pipeline = FillMaskPipeline(model, tokenizer_hf)
    res_hf = pipeline(seed)
    max_score_sequence = max(res_hf, key=lambda x: x['score'])['sequence']
    seed = max_score_sequence + ' <mask>'
    i -= 1

print(seed)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, set_seed
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
model = AutoModelForCausalLM.from_pretrained("PlanTL-GOB-ES/gpt2-large-bne")
generator = pipeline('text-generation', tokenizer=tokenizer, model=model)
set_seed(42)
generator("Los  ", num_return_sequences=5)

# Zero-shot experiment

In [172]:
from utils.constants import STAGE_DIR

with open(STAGE_DIR / 'previous-work' / 'EVAL_DATA.txt') as f:
    sentences = f.read()

In [173]:
from utils.functions import get_dict_from_data

evaluation_data = get_dict_from_data(sentences)

In [174]:
evaluation_data

[{'text': 'Los trabajos realizados a título de amistad, benevolencia o buena vecindad.',
  'subject': 'vecindad',
  'object': 'trabajos',
  'event': 'realizados',
  'complement': None,
  'subjectLabel': 'LegalConcept',
  'objectLabel': 'LegalConcept',
  'relationType': 'Norelation'},
 {'text': 'Las situaciones de incapacidad temporal, nacimiento, adopción, guarda con fines de adopción, acogimiento, riesgo durante el embarazo, riesgo durante la lactancia y violencia de género interrumpirán el cómputo de la duración del contrato.',
  'subject': 'contrato',
  'object': None,
  'event': 'interrumpirán',
  'complement': None,
  'subjectLabel': 'LegalConcept',
  'objectLabel': 'LegalConcept',
  'relationType': 'Norelation'},
 {'text': 'Si el trabajador tuviera asignadas condiciones o retribuciones especiales en virtud de contraprestaciones establecidas en la parte no válida del contrato, el órgano de la jurisdicción social que a instancia de parte declare la nulidad hará el debido pronunciam

In [175]:
sentences = "\n\n".join(sentence_data['text'] for sentence_data in evaluation_data)

In [130]:
prompt = f'''
We are interested in extracting event information from spanish legal text. Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger following the next definitions.
    * subject: Agent of the action, who performs the action.
    * event trigger: Action
    * object: Receiver of the action.
3. Classify each subject in one of the following labels:
    * LegalAgent: Natural person
    * LegalEntity: Not natural person nor individual. Normally a corporation or an enterprise
    * LegalConcept: Not natural person nor corporation.
4. Classify each object entity in one of the labels of previous step.
5. Classify the relation in one of the following classes:
    * Right
    * Duty
    * NoRight
    * Priviledge
    * Norelation
Right and No-Right are opposites and Duty and Priviledge are opposite as well. Norelation represents a sentence that does not contains a relation.
6. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    * sentence: the sentence
    * subject: the subject entity
    * object: the object entity
    * event: the event trigger
    * subjectLabel: the classification of the subject entity in step 3
    * objectLabel: the classification of the object entity in step 4
    * relationType: the classification of the relation in step 5

The set of sentences in spanish to use is the following:

```
{sentences}
```
'''

In [122]:
print(prompt)


We are interested in extracting event information from spanish legal text. Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity, an event trigger and a complement following the next definitions.
    * subject: Agent of the action, who performs the action.
    * event trigger: Action
    * object: Receiver of or entity affected by the action.
3. Classify each subject in one of the following labels:
    * LegalAgent: Natural person
    * LegalEntity: Not natural person nor individual. Normally a corporation or an enterprise
    * LegalConcept: Not natural person nor corporation.
4. Classify each object entity in one of the labels of previous step.
5. Classify the relation in one of the following classes:
    * Right
    

In [131]:
from utils.functions import get_completion

response = get_completion(prompt)

In [132]:
print(response)

[
    {
        "sentence": "Los trabajos realizados a título de amistad, benevolencia o buena vecindad.",
        "subject": "trabajos",
        "object": null,
        "event": "realizados",
        "subjectLabel": "LegalConcept",
        "objectLabel": null,
        "relationType": "Norelation"
    },
    {
        "sentence": "Las situaciones de incapacidad temporal, nacimiento, adopción, guarda con fines de adopción, acogimiento, riesgo durante el embarazo, riesgo durante la lactancia y violencia de género interrumpirán el cómputo de la duración del contrato.",
        "subject": "situaciones",
        "object": "cómputo de la duración del contrato",
        "event": "interrumpirán",
        "subjectLabel": "LegalConcept",
        "objectLabel": "LegalConcept",
        "relationType": "NoRight"
    },
    {
        "sentence": "Si el trabajador tuviera asignadas condiciones o retribuciones especiales en virtud de contraprestaciones establecidas en la parte no válida del contrato, 

In [133]:
import json

evaluation_predict = json.loads(response)

In [ ]:
evaluation_predict

In [138]:
from utils.metrics import match_score

match_score(evaluation_data, evaluation_predict, 'exact')

,Subject,Object,Event,Total
COR,5.000000,1.0,7.000000,13.000000
INC,6.000000,6.0,6.000000,18.000000
PAR,0.000000,0.0,0.000000,0.000000
MIS,2.000000,3.0,0.000000,5.000000
SPU,0.000000,3.0,0.000000,3.000000
Precision,0.454545,0.1,0.538462,0.382353
Recall,0.384615,0.1,0.538462,0.361111
F1 Score,0.416667,0.1,0.538462,0.371429


In [143]:
match_score(evaluation_data, evaluation_predict, 'partial')

,Subject,Object,Event,Total
COR,5.000000,1.0,7.000000,13.000000
INC,6.000000,6.0,5.000000,17.000000
PAR,0.000000,0.0,1.000000,1.000000
MIS,2.000000,3.0,0.000000,5.000000
SPU,0.000000,3.0,0.000000,3.000000
Precision,0.454545,0.1,0.576923,0.397059
Recall,0.384615,0.1,0.576923,0.375000
F1 Score,0.416667,0.1,0.576923,0.385714


In [140]:
match_score(evaluation_data, evaluation_predict, 'strict')

,Subject,Object,Event,Total
COR,4.000000,0.0,4.000000,8.000000
INC,7.000000,7.0,9.000000,23.000000
PAR,0.000000,0.0,0.000000,0.000000
MIS,2.000000,3.0,0.000000,5.000000
SPU,0.000000,3.0,0.000000,3.000000
Precision,0.363636,0.0,0.307692,0.235294
Recall,0.307692,0.0,0.307692,0.222222
F1 Score,0.333333,NaN,0.307692,0.228571


In [141]:
match_score(evaluation_data, evaluation_predict, 'type')

,Subject,Object,Event,Total
COR,4.000000,0.0,4.000000,8.000000
INC,8.000000,12.0,9.000000,29.000000
PAR,0.000000,0.0,0.000000,0.000000
MIS,1.000000,1.0,0.000000,2.000000
SPU,0.000000,0.0,0.000000,0.000000
Precision,0.333333,0.0,0.307692,0.216216
Recall,0.307692,0.0,0.307692,0.205128
F1 Score,0.320000,NaN,0.307692,0.210526


In [144]:
from utils.metrics import compute_average_similarity_score

print(f'''
Subject - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'subject')}
Object - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'object')}
Trigger - {compute_average_similarity_score(evaluation_data, evaluation_predict, 'event')}
''')



Subject - 0.4230807787106761
Object - 0.14261991702024798
Trigger - 0.6474204001127077



# Few-shot experiment

In [145]:
train_data = evaluation_data[:3]
few_evaluation_data = evaluation_data[3:]
few_sentences = '\n\n'.join(s['text'] for s in few_evaluation_data)

In [ ]:
subject_train_data = "\n".join(f"`{s['text']}` -> {s['subject']}" for s in train_data)
object_train_data = "\n".join(f"`{s['text']}` -> {s['object']}" for s in train_data)
event_train_data = "\n".join(f"`{s['text']}` -> {s['event']}" for s in train_data)
complement_train_data = "\n".join(f"`{s['text']}` -> {s['complement']}" for s in train_data)

In [157]:
prompt = f'''
We are interested in extracting event information from spanish legal text and you are a system that will compute that task for us.

Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger following the next definitions.
    * subject: Agent of the action, who performs the action.
    * event trigger: Action
    * object: Receiver of the action.
3. Classify each subject in one of the following labels:
    * LegalAgent: Natural person
    * LegalEntity: Not natural person nor individual. Normally a corporation or an enterprise
    * LegalConcept: Not natural person nor corporation.
4. Classify each object entity in one of the labels of previous step.
5. Classify the relation in one of the following classes:
    * Right
    * Duty
    * NoRight
    * Priviledge
    * Norelation
Right and No-Right are opposites and Duty and Priviledge are opposite as well. Norelation represents a sentence that does not contains a relation.
6. The output of the task should be a list of dictionaries, each dictionary contains the following keys:
    * text: the sentence
    * subject: the subject entity
    * object: the object entity
    * event: the event trigger
    * subjectLabel: the classification of the subject entity in step 3
    * objectLabel: the classification of the object entity in step 4
    * relationType: the classification of the relation in step 5

To use it as a guide, between triple dashes you are going to find examples of sentences and the desired output:

---
Example 1:
* Sentence: {train_data[0]["text"]}
* Output:
{train_data[0]}

Example 2:
* Sentence: {train_data[1]["text"]}
* Output:
{train_data[1]}

Example 3:
* Sentence: {train_data[2]["text"]}
* Output:
{train_data[2]}
---

The set of sentences in spanish to use in the event extraction task is the following:

```
{few_sentences}
```
'''

In [158]:
print(prompt)


We are interested in extracting event information from spanish legal text and you are a system that will compute that task for us.

Given a large set of sentences in spanish from the legal domain, written between triple backticks, your objective is to extract the roles from the text written in Spanish following the next steps:
1. Identify each sentence in the corpus separated for new lines.
2. In each sentence detect a subject entity, an object entity and an event trigger following the next definitions.
    * subject: Agent of the action, who performs the action.
    * event trigger: Action
    * object: Receiver of the action.
3. Classify each subject in one of the following labels:
    * LegalAgent: Natural person
    * LegalEntity: Not natural person nor individual. Normally a corporation or an enterprise
    * LegalConcept: Not natural person nor corporation.
4. Classify each object entity in one of the labels of previous step.
5. Classify the relation in one of the following clas

In [159]:
few_response = get_completion(prompt)

In [160]:
print(few_response)

[
    {
        "text": "En caso de nulidad por discriminación salarial por razón de sexo, el trabajador tendrá derecho a la retribución correspondiente al trabajo igual o de igual valor.",
        "subject": "trabajador",
        "object": null,
        "event": "tendrá derecho",
        "subjectLabel": "LegalAgent",
        "objectLabel": "LegalConcept",
        "relationType": "Right"
    },
    {
        "text": "Si el empresario diera un trabajo en común a un grupo de sus trabajadores, conservará respecto de cada uno, individualmente, sus derechos y deberes.",
        "subject": "empresario",
        "object": "trabajadores",
        "event": "conservará",
        "subjectLabel": "LegalAgent",
        "objectLabel": "LegalEntity",
        "relationType": "Duty"
    },
    {
        "text": "El jefe del grupo ostentará la representación de los que lo integren, respondiendo de las obligaciones inherentes a dicha representación.",
        "subject": "jefe del grupo",
        "object"

In [161]:
few_evaluation_predict = json.loads(few_response)

In [162]:
match_score(few_evaluation_data, few_evaluation_predict, 'exact')

,Subject,Object,Event,Total
COR,4.000000,3.000000,7.0,14.000000
INC,5.000000,5.000000,3.0,13.000000
PAR,0.000000,0.000000,0.0,0.000000
MIS,0.000000,0.000000,0.0,0.000000
SPU,1.000000,2.000000,0.0,3.000000
Precision,0.400000,0.300000,0.7,0.466667
Recall,0.444444,0.375000,0.7,0.518519
F1 Score,0.421053,0.333333,0.7,0.491228


In [163]:
match_score(few_evaluation_data, few_evaluation_predict, 'partial')

,Subject,Object,Event,Total
COR,4.000000,3.000000,7.00,14.000000
INC,5.000000,5.000000,2.00,12.000000
PAR,0.000000,0.000000,1.00,1.000000
MIS,0.000000,0.000000,0.00,0.000000
SPU,1.000000,2.000000,0.00,3.000000
Precision,0.400000,0.300000,0.75,0.483333
Recall,0.444444,0.375000,0.75,0.537037
F1 Score,0.421053,0.333333,0.75,0.508772


In [164]:
match_score(few_evaluation_data, few_evaluation_predict, 'strict')

,Subject,Object,Event,Total
COR,4.000000,0.0,3.0,7.000000
INC,5.000000,8.0,7.0,20.000000
PAR,0.000000,0.0,0.0,0.000000
MIS,0.000000,0.0,0.0,0.000000
SPU,1.000000,2.0,0.0,3.000000
Precision,0.400000,0.0,0.3,0.233333
Recall,0.444444,0.0,0.3,0.259259
F1 Score,0.421053,NaN,0.3,0.245614


In [165]:
match_score(few_evaluation_data, few_evaluation_predict, 'type')

,Subject,Object,Event,Total
COR,4.0,0.0,3.0,7.000000
INC,6.0,10.0,7.0,23.000000
PAR,0.0,0.0,0.0,0.000000
MIS,0.0,0.0,0.0,0.000000
SPU,0.0,0.0,0.0,0.000000
Precision,0.4,0.0,0.3,0.233333
Recall,0.4,0.0,0.3,0.233333
F1 Score,0.4,NaN,0.3,0.233333


In [166]:
from utils.metrics import compute_average_similarity_score

print(f'''
Subject - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'subject')}
Object - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'object')}
Trigger - {compute_average_similarity_score(few_evaluation_data, few_evaluation_predict, 'event')}
''')



Subject - 0.4441071428571429
Object - 0.40115476190476185
Trigger - 0.853409090909091

